In [ ]:
import pandas as pd
from tqdm import tqdm
from gensim.models import Word2Vec
from datetime import datetime

In [ ]:
df_x = pd.read_csv("../dataset/dressipi_recsys2022/train_sessions.csv")
df_y = pd.read_csv("../dataset/dressipi_recsys2022/train_purchases.csv")

In [ ]:
df_x['session_id'] = df_x['session_id'].astype(str)
df_x['item_id'] = df_x['item_id'].astype(str)
df_y['session_id'] = df_y['session_id'].astype(str)
df_y['item_id'] = df_y['item_id'].astype(str)

In [ ]:
df_x.sort_values(by=['date'],inplace=True)

In [ ]:
df_x.head()

,session_id,item_id,date,formatted_date
4005618,3747794,11234,2020-01-01 00:00:01.359,2020-01
3699175,3458777,3868,2020-01-01 00:00:21.44,2020-01
4005616,3747794,9214,2020-01-01 00:00:22.294,2020-01
4649622,4350716,10861,2020-01-01 00:00:48.505,2020-01
4005614,3747794,27092,2020-01-01 00:02:14.538,2020-01


In [ ]:
df_x['formatted_date'] = df_x['date'].apply(lambda x:x[:7])

In [ ]:
xf = df_x['formatted_date'].value_counts().to_frame().reset_index()

In [ ]:
xf.rename(columns={'index':'month','formatted_data':'count'},inplace=True)
xf.sort_values(by=['month'],inplace=True)

In [ ]:
xf['cumsum'] = xf['formatted_date'].cumsum()
xf['cum_per'] = xf['cumsum'] / xf['cumsum'].tolist()[-1]

In [ ]:
xf

,month,formatted_date,cumsum,cum_per
13,2020-01,182730,182730,0.038520
16,2020-02,133741,316471,0.066712
14,2020-03,174761,491232,0.103552
9,2020-04,267263,758495,0.159891
12,2020-05,204240,962735,0.202945
8,2020-06,283704,1246439,0.262750
11,2020-07,240108,1486547,0.313365
6,2020-08,296736,1783283,0.375917
2,2020-09,373697,2156980,0.454693
7,2020-10,284919,2441899,0.514754


In [ ]:
test_months = xf[xf['cum_per'] > 0.91]['month']

In [ ]:
test_months

0    2021-05
Name: month, dtype: object

In [ ]:
test_df = df_x[df_x['formatted_date'].isin(test_months)]
train_df = df_x[~df_x['formatted_date'].isin(test_months)]

In [ ]:
df_y['is_train'] = df_y['session_id'].isin(train_df['session_id'])

In [ ]:
train_y = df_y[df_y['is_train']==True]
test_y = df_y[df_y['is_train']==False]

In [ ]:
len(train_df['session_id'].unique().tolist()), len(test_df['session_id'].unique().tolist()), 

(918382, 81618)

In [ ]:
train_df.to_csv("../dataset/data/x_train.csv",index=False)
test_df.to_csv("../dataset/data/x_test.csv",index=False)
train_y.to_csv("../dataset/data/y_train.csv",index=False)
test_y.to_csv("../dataset/data/y_test.csv",index=False)